# UFABC PDF to Spreadsheet

Define the ```url``` and ```file_name``` string variables that contain the URL of the PDF file to be converted and the new files name, repectively.

In [2]:
url = "http://prograd.ufabc.edu.br/pdf/turmas_salas_docentes_sa_2018.1.pdf"
file_name = "2018.1_SA" # without extention

Import libraries needed to display files.

In [ ]:
from pathlib import Path
from IPython.display import FileLink
from IPython.display import IFrame

Download the PDF file.

In [3]:
file_name_pdf = file_name + '.pdf'
!wget $url -O $file_name_pdf
file_pdf = FileLink(file_name_pdf)
!echo PDF file saved as:
display(file_pdf)
IFrame(file_name_pdf, width=600, height=300)

--2018-02-18 13:54:57--  http://prograd.ufabc.edu.br/pdf/turmas_salas_docentes_sa_2018.1.pdf
Resolving prograd.ufabc.edu.br (prograd.ufabc.edu.br)... 200.133.215.63
Connecting to prograd.ufabc.edu.br (prograd.ufabc.edu.br)|200.133.215.63|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689670 (674K) [application/pdf]
Saving to: ‘2018.1_SA.pdf’

2018.1_SA.pdf       100%[===================>] 673.51K   256KB/s    in 2.6s    

2018-02-18 13:55:00 (256 KB/s) - ‘2018.1_SA.pdf’ saved [689670/689670]

PDF file saved as:


/home/jovyan/2018.1_SA.pdf

The PDF file is converted to a CSV file (*this might take a while*).

In [ ]:
file_name_csv = file_name + ".csv"
!java -Dfile.encoding=utf-8 -jar tabula.jar -l --pages 3 $file_name_pdf -o $file_name_csv
file_csv = FileLink(file_name_csv)
!echo CSV file saved as:
display(file_csv)

Configure and simplify the Natural Language Toolkit (NTLK) Portuguese treebank.

In [5]:
import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
nltk.download('floresta')
from nltk import tokenize
from nltk.corpus import floresta
def simplify_tag(t):
    if "+" in t:
        return t[t.index("+")+1:]
    else:
        return t
twords = nltk.corpus.floresta.tagged_words()
twords = [(w.lower(),simplify_tag(t)) for (w,t) in twords]

# Insert some missing prepositions
twords.insert(0,('da','prp'))
twords.insert(0,('de','prp'))
twords.insert(0,('di','prp'))
twords.insert(0,('do','prp'))
twords.insert(0,('du','prp'))

ModuleNotFoundError: No module named 'nltk'

With NTLK propely prepared, create a ```title_pos_tag``` function that imitates ```title``` built-in function but doesn't capitalize conjunctions and prepositions. It is useful when titling proper names.

In [7]:
def title_pos_tag(text):
    def pos_tag_portuguese(tokens):
        for index in range(len(tokens)):
            for word in twords:
                token = tokens[index].lower()
                if word[0] == token:
                    tag = word[1]
                    tokens[index] = (token, tag)
                    break
        return tokens
    tokens = tokenize.word_tokenize(text, language='portuguese')
    tagged = pos_tag_portuguese(tokens)
    new_text = ''
    for index in range(len(tagged)):
        token = tagged[index]
        if isinstance(token, tuple):
            word = token[0]
            tag  = token[1]
            # n:         substantivo
            # prop:      nome próprio
            # art:       artigo
            # pron:      pronome
            # pron-pers: pronome pessoal
            # pron-det:  pronome determinativo
            # pron-indp: substantivo/pron-indp
            # adj:       adjetivo
            # n-adj:     substantivo/adjetivo
            # v:         verbo
            # v-fin:     verbo finitivo
            # v-inf:     verbo infinitivo
            # v-pcp:     verbo particípio
            # v-ger:     verbo gerúndio
            # num:       numeral
            # prp:       preposição
            # adj:       adjetivo
            # conj:      conjunção
            # conj-s:    conjunção subordinativa
            # conj-c:    conjunção coordenativa
            # intj:      interjeição
            # adv:       advérbio
            # xxx:       outro
            if 'conj' in tag or \
               'prp'  in tag:
                new_text = new_text + ' ' + word.lower()
            else:
                new_text = new_text + ' ' + word.capitalize()
        else:
            new_text = new_text + ' ' + token.capitalize()
    new_text = new_text.strip()
#     return (new_text, tagged) # uncomment this line if is desired to retriev the tags as well
    return new_text

Create an object that shows expandable JSON files (*credits to David Caldwell*)

In [6]:
import uuid
from IPython.display import HTML
# from IPython.display import display_javascript, display_html#, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        htmlstr = """
        <html>
            <head>
                <style>body {background-color:yellow;}</style>
            </head>
            <body>
                <div id="{0}" style="height: 600px; width:100%;"></div>
                <script>
                    require(["renderjson.js"], function() {
                        document.getElementById('{0}').appendChild(renderjson({1}))
                    });
                </script>
            </body>
        </html>
        """.format(self.uuid, self.json_str)
        HTML(htmlstr)
#         display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid), raw=True)
#         display_javascript("""
#         require(["renderjson.js"], function() {
#             document.getElementById('%s').appendChild(renderjson(%s))
#         });
#         """ % (self.uuid, self.json_str), raw=True)

The CSV file is then processed into a JSON file

In [7]:
file_name_json = file_name + '.json'
import csv
with open(file_name_csv, encoding="utf-8") as csv_file:
    full_data = []
    content = csv.reader(csv_file, delimiter=',', quotechar='"')
    week_names = ('segunda','terça','quarta','quinta','sexta','sábado','domingo')
    index = -1
    for row in content:
        index = index + 1
        if index:
#             print(', '.join(row).replace('\r',''))
#             print()
            column = 0
            for cell in row:
                column = column + 1
                data = cell.replace('\r','').replace('\n',' ').replace(' , ',', ').strip()
                if   data == '¬': data = ''
                elif data == '0': data = ''

                # Código
                if column == 1:
                    codigo = data.upper()

                # Disciplina - turma
                elif column == 2:
                    # Campus
                    data, _, campus = data.rpartition('(')
                    campus = title_pos_tag(campus[:-1])

                    # Disciplina
                    disciplina, _, data = data.strip().rpartition(' ')
                    disciplina = title_pos_tag(disciplina)

                    # Turma e período
                    turma, _, periodo = data.strip().rpartition('-')
                    turma   = turma.upper()
                    periodo = periodo.capitalize()
                    
                    # Subcódigo
                    subcodigo, _, _ = codigo.partition('-')
                    subcodigo = subcodigo[len(turma)+1:]


                # Teoria
                elif column == 3:
                    for week in week_names:
                        data = data.replace(week, '\n' + week)
                    teoria = data.replace(', \n','\n').strip().splitlines()
                    
                    teoria_num_of_days = len(teoria)
                    teoria_dia_da_semana = [None]*teoria_num_of_days
                    teoria_entrada       = [None]*teoria_num_of_days
                    teoria_saida         = [None]*teoria_num_of_days
                    teoria_sala          = [None]*teoria_num_of_days
                    teoria_frequencia    = [None]*teoria_num_of_days
                    for day in range(teoria_num_of_days):
                        data = teoria[day]
                        teoria_dia_da_semana[day], _, data                   = data.partition(' das ')
                        teoria_entrada[day],       _, data                   = data.partition(' às ')
                        teoria_saida[day],         _, data                   = data.partition(', sala ')
                        teoria_sala[day],          _, teoria_frequencia[day] = data.partition(', ')
                        
                        teoria_dia_da_semana[day] = teoria_dia_da_semana[day].capitalize()
                        teoria_frequencia[day]    = teoria_frequencia[day].capitalize()
                        teoria_sala[day]          = teoria_sala[day].upper()

                # Prática
                elif column == 4:
                    for week in week_names:
                        data = data.replace(week, '\n' + week)
                    pratica = data.replace(',\n','\n').strip().splitlines()
                    
                    pratica_num_of_days = len(pratica)
                    pratica_dia_da_semana = [None]*pratica_num_of_days
                    pratica_entrada       = [None]*pratica_num_of_days
                    pratica_saida         = [None]*pratica_num_of_days
                    pratica_sala          = [None]*pratica_num_of_days
                    pratica_frequencia    = [None]*pratica_num_of_days
                    for day in range(pratica_num_of_days):
                        data = pratica[day]
                        pratica_dia_da_semana[day], _, data                   = data.partition(' das ')
                        pratica_entrada[day],       _, data                   = data.partition(' às ')
                        pratica_saida[day],         _, data                   = data.partition(', sala ')
                        pratica_sala[day],          _, pratica_frequencia[day] = data.partition(', ')
                        
                        pratica_dia_da_semana[day] = pratica_dia_da_semana[day].capitalize()
                        pratica_frequencia[day]    = pratica_frequencia[day].capitalize()
                        pratica_sala[day]          = pratica_sala[day].upper()

                # Docente teoria
                elif column == 5:
                    docente_teoria = title_pos_tag(data)

                # Docente prática
                elif column == 6:
                    docente_pratica = title_pos_tag(data)

            teoria = []
            i = 0
            for day in range(teoria_num_of_days):
                i = i + 1
                teoria_new = {'id': i,
                              'dia_da_semana': teoria_dia_da_semana[day],
                              'horario_de_entrada': teoria_entrada[day],
                              'horario_de_saida': teoria_saida[day],
                              'sala': teoria_sala[day],
                              'frequencia': teoria_frequencia[day]}
                teoria.append(teoria_new)
                
            pratica = []
            i = -1
            for day in range(pratica_num_of_days):
                i = i + 1
                pratica_new = {'id': i,
                               'dia_da_semana': pratica_dia_da_semana[day],
                               'horario_de_entrada': pratica_entrada[day],
                               'horario_de_saida': pratica_saida[day],
                               'sala': pratica_sala[day],
                               'frequencia': pratica_frequencia[day]}
                pratica.append(pratica_new)
                
            new_data = {'id': index-1,
                        'codigo': codigo,
                        'subcodigo': subcodigo,
                        'disciplina': disciplina,
                        'campus': campus,
                        'periodo': periodo,
                        'turma': turma,
                        'teoria': teoria,
                        'pratica': pratica,
                        'docente_teoria': docente_teoria,
                        'docente_pratica': docente_pratica}
            full_data.append(new_data)
    with open(file_name_json, 'w') as file:
        json.dump(full_data, file)
        file_json = FileLink(file_name_json)
        !echo JSON file saved as:
        display(file_json)
    with open(file_name_json, 'r') as file:
        data = json.load(file)
        RenderJSON(data)

FileNotFoundError: [Errno 2] No such file or directory: '2018.1_SA.csv'

JSON preview

Process the JSON file into a spreadsheet

In [12]:
with open(file_name_json, 'r') as file:
    data = json.load(file)
#     print(data[0]['codigo'])
    for disciplina in data:
        print(disciplina['codigo'])

DAESZM035-17SA
NA1MCTB001-17SA
DAMCTB001-17SA
NAMCTB001-17SA
DA1MCTA001-17SA
NA1MCTA001-17SA
DA2MCTA001-17SA
NA2MCTA001-17SA
DB1MCTA001-17SA
DB2MCTA001-17SA
DAMCZA035-14SA


In [8]:
import numpy as np
import pandas as pd
import qgrid
randn = np.random.randn
df_types = pd.DataFrame({
    'A' : pd.Series(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06', '2013-01-07', '2013-01-08', '2013-01-09'],index=list(range(9)),dtype='datetime64[ns]'),
    'B' : pd.Series(randn(9),index=list(range(9)),dtype='float32'),
    'C' : pd.Categorical(["washington", "adams", "washington", "madison", "lincoln","jefferson", "hamilton", "roosevelt", "kennedy"]),
    'D' : ["foo", "bar", "buzz", "bippity","boppity", "foo", "foo", "bar", "zoo"] })
df_types['E'] = df_types['D'] == 'foo'
qgrid_widget = qgrid.QgridWidget(df=df_types, show_toolbar=True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8}, precision=5, show_toolbar=True)